In [1]:
import pandas as pd
import numpy as np

In [2]:
# importar pacotes usados na seleção do modelo e na medição da precisão
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

In [3]:
df = pd.read_csv('afastamentos-nomes-sexo.csv', index_col=0)

In [4]:
df.drop(columns=['REV'], inplace=True)

In [5]:
df.head(10)

,QTDE,SEXO,LETRAS,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13
PNOME,,,,,,,,,,,,,,,,,
MARIA,2937,F,5,A,I,R,A,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JOSE,2766,M,4,E,S,O,J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANTONIO,1257,M,7,O,I,N,O,T,N,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FRANCISCO,1014,M,9,O,C,S,I,C,N,A,R,F,NaN,NaN,NaN,NaN,NaN
JOAO,993,M,4,O,A,O,J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CARLOS,979,M,6,S,O,L,R,A,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANA,938,F,3,A,N,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PAULO,907,M,5,O,L,U,A,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LUIZ,829,M,4,Z,I,U,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df['SEXO'] = df['SEXO'].map({'F': 0, 'M': 1})
df.loc[df['SEXO'].isnull(), 'SEXO'] = 2
df['SEXO'] = df['SEXO'].astype('uint8')

In [7]:
for i in range(14):
    col = 'L' + str(i)
    df.loc[df[col].isnull(), col] = ' '
    df[col] = df[col].apply(lambda x: ord(x) - 64 if x != ' ' else 0).astype('uint8')

In [8]:
#df.dtypes

In [9]:
df.head(10)

,QTDE,SEXO,LETRAS,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13
PNOME,,,,,,,,,,,,,,,,,
MARIA,2937,0,5,1,9,18,1,13,0,0,0,0,0,0,0,0,0
JOSE,2766,1,4,5,19,15,10,0,0,0,0,0,0,0,0,0,0
ANTONIO,1257,1,7,15,9,14,15,20,14,1,0,0,0,0,0,0,0
FRANCISCO,1014,1,9,15,3,19,9,3,14,1,18,6,0,0,0,0,0
JOAO,993,1,4,15,1,15,10,0,0,0,0,0,0,0,0,0,0
CARLOS,979,1,6,19,15,12,18,1,3,0,0,0,0,0,0,0,0
ANA,938,0,3,1,14,1,0,0,0,0,0,0,0,0,0,0,0
PAULO,907,1,5,15,12,21,1,16,0,0,0,0,0,0,0,0,0
LUIZ,829,1,4,26,9,21,12,0,0,0,0,0,0,0,0,0,0


In [10]:
#df.describe()

In [11]:
# definir dados de entrada

X = df.drop(['SEXO'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo

print('Forma dos dados originais:', X.shape, y.shape)

Forma dos dados originais: (9726, 16) (9726,)


In [12]:
X.head()

,QTDE,LETRAS,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13
PNOME,,,,,,,,,,,,,,,,
MARIA,2937,5,1,9,18,1,13,0,0,0,0,0,0,0,0,0
JOSE,2766,4,5,19,15,10,0,0,0,0,0,0,0,0,0,0
ANTONIO,1257,7,15,9,14,15,20,14,1,0,0,0,0,0,0,0
FRANCISCO,1014,9,15,3,19,9,3,14,1,18,6,0,0,0,0,0
JOAO,993,4,15,1,15,10,0,0,0,0,0,0,0,0,0,0


In [13]:
y.head()

PNOME
MARIA        0
JOSE         1
ANTONIO      1
FRANCISCO    1
JOAO         1
Name: SEXO, dtype: uint8

In [14]:
NUMBER_KFOLD_SPLITS = 5 # number of splits in cross-validation
NUMBER_GRID_ITERATIONS = 10 # number of grid iterations to parameters testing
SCORING_METRIC = 'accuracy' # the scoring metric to be used

In [15]:
from datetime import datetime

models = {}

def evaluate_model(name, model, X=X, y=y):

  start = datetime.now()
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)
  results = cross_val_score(model, X, y, cv=kfold,
                            scoring=SCORING_METRIC, verbose=1, n_jobs=-1)
  end = datetime.now()

  elapsed = int((end - start).total_seconds() * 1000)
  score = results.mean() * 100
  stddev = results.std() * 100

  models[name] = (model, score, stddev, elapsed)
  print(model, '\nCross-Validation Score: %.2f (+/- %.2f) [%5s ms]' % \
        (score, stddev, elapsed))
  return score, stddev, elapsed

In [16]:
# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):

  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)

  search = RandomizedSearchCV(model, param_distributions=params, 
                              n_iter=NUMBER_GRID_ITERATIONS, 
                              scoring=SCORING_METRIC, cv=kfold, 
                              verbose=1, n_jobs=-1)
  #search = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=kfold, verbose=1)

  search.fit(X, y)
  print('\nBest Score: %.2f %%' % (search.best_score_ * 100))
  print('Best Params:', search.best_params_)
  return search

In [17]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42, multi_class='auto', max_iter=1000, solver='liblinear', C=0.1)
evaluate_model('LR', model)

params = dict(solver=['liblinear','lbfgs'], C=np.logspace(-3,3,7))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LogisticRegression(C=0.1, max_iter=1000, random_state=42, solver='liblinear') 
Cross-Validation Score: 85.16 (+/- 0.76) [ 3531 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.5s finished


In [18]:
# Support Vector Machine (SVM)
from sklearn.svm import SVC

model = SVC(random_state=42, C=10, gamma=0.001) # 85.29
evaluate_model('SVM', model)

#params = dict(C=np.logspace(-1,1,3)) # 87.90
#params = dict(gamma=np.logspace(-3,0,4)) # 89.46
#params = dict(kernel=['linear','rbf','poly','sigmoid']) # 
params = dict(C=np.logspace(-1,1,3),
              gamma=np.logspace(-3,0,4),
              kernel=['linear','rbf','poly','sigmoid'])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


SVC(C=10, gamma=0.001, random_state=42) 
Cross-Validation Score: 89.46 (+/- 0.31) [12446 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   12.4s finished


In [19]:
# Linear SVM
from sklearn.svm import LinearSVC

model = LinearSVC(random_state=42, max_iter=1000, C=0.001)
evaluate_model('LSVM', model)

#from scipy.stats import reciprocal, uniform
#params = dict(C=uniform(1, 10))
params = dict(C=np.logspace(-4,3,8))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearSVC(C=0.001, random_state=42) 
Cross-Validation Score: 85.26 (+/- 0.79) [ 7046 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    7.0s finished


In [20]:
# K-Nearest Neighbours (KNN)
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=9)
evaluate_model('KNN', model)

params = dict(n_neighbors=[1,3,5,7,9,11])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KNeighborsClassifier(n_neighbors=9) 
Cross-Validation Score: 87.47 (+/- 0.50) [ 3358 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.4s finished


In [21]:
# Linear Discriminant Analysis (LDA)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis(solver='svd')
evaluate_model('LDA', model)

params = dict(solver=['svd','lsqr','eigen'])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearDiscriminantAnalysis() 
Cross-Validation Score: 84.42 (+/- 0.69) [  464 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished


In [22]:
# Quadratic Discriminant Analysis (QDA)
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

model = QuadraticDiscriminantAnalysis()
evaluate_model('QDA', model)

params = dict()
#fine_tune_model(model, params)

QuadraticDiscriminantAnalysis() 
Cross-Validation Score: 44.99 (+/- 0.74) [  155 ms]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


In [23]:
# Naïve Bayes - Gaussian
from sklearn.naive_bayes import GaussianNB

model = GaussianNB(priors=None, var_smoothing=0.1)
evaluate_model('GNB', model)

params = dict(var_smoothing=np.logspace(0,-9,num=10))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


GaussianNB(var_smoothing=0.1) 
Cross-Validation Score: 67.58 (+/- 11.38) [  122 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


In [24]:
# Naïve Bayes - Bernoulli
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()
evaluate_model('BNB', model)

params = dict(var_smoothing=np.logspace(0,-9,num=10))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


BernoulliNB() 
Cross-Validation Score: 43.36 (+/- 2.19) [  128 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


In [25]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=17)
evaluate_model('DT', model)

params = dict(criterion=['gini','entropy'], max_depth=[3,5,7,11,13,17,19])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


DecisionTreeClassifier(criterion='entropy', max_depth=17, random_state=42) 
Cross-Validation Score: 96.51 (+/- 0.63) [  199 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


In [26]:
# Ensemble - Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42, max_features='log2', n_estimators=500)
evaluate_model('RF', model)

params = dict(n_estimators=[10,50,100,500], max_features=['auto','sqrt','log2'])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


RandomForestClassifier(max_features='log2', n_estimators=500, random_state=42) 
Cross-Validation Score: 94.00 (+/- 0.42) [21692 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   21.7s finished


In [27]:
# Ensemble - Ada Boost
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=17), n_estimators=13)
evaluate_model('ABDT', model)

params = dict(n_estimators=[1,3,5,7,9,11,13])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                         max_depth=17,
                                                         random_state=42),
                   n_estimators=13) 
Cross-Validation Score: 96.43 (+/- 0.68) [  216 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


In [28]:
# Ensemble - Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=42, max_depth=35, n_estimators=250, learning_rate=0.1)
evaluate_model('GB', model)

params = dict(max_depth=[1,3,5,7,9], n_estimators=[5,50,250,500], learning_rate=[0.01,0.1,1,10,100])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


GradientBoostingClassifier(max_depth=35, n_estimators=250, random_state=42) 
Cross-Validation Score: 96.28 (+/- 0.42) [225645 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.8min finished


In [29]:
# Stochastic Gradient Descent (SGD)
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(random_state=42, max_iter=500, tol=0.1)
evaluate_model('SGD', model)

params = dict(max_iter=[100, 200, 350, 500, 1000], tol=[0.01, 0.1, 1.0])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


SGDClassifier(max_iter=500, random_state=42, tol=0.1) 
Cross-Validation Score: 72.85 (+/- 9.75) [  546 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished


In [30]:
# Perceptron
from sklearn.linear_model import Perceptron

model = Perceptron(random_state=42, max_iter=500, tol=0.001)
evaluate_model('PCT', model)

params = dict(max_iter=[100, 200, 350, 500, 750, 1000], tol=[0.1, 0.01, 0.001])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Perceptron(max_iter=500, random_state=42) 
Cross-Validation Score: 70.79 (+/- 6.99) [  265 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished


In [31]:
# Multi-Layer Perceptron (MLP)
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=42, solver='lbfgs', alpha=1e-07, hidden_layer_sizes=(50, 100, 50))
evaluate_model('MLP', model)

params = dict(alpha=np.logspace(-9,-1,5), 
              solver=['lbfgs','sgd','adam'], 
              hidden_layer_sizes=[(50,50,50), (50,100,50), (100,)])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


MLPClassifier(alpha=1e-07, hidden_layer_sizes=(50, 100, 50), random_state=42,
              solver='lbfgs') 
Cross-Validation Score: 88.97 (+/- 0.54) [59658 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   59.7s finished


In [32]:
# Extreme Gradient Boosting Machine (XGBM)
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=9, min_child_weight=1, gamma=0.1, subsample=0.9, colsample_bytree=0.7, reg_alpha=1e-05)
evaluate_model('XGB', model)

params = dict(max_depth=range(3,10,2), min_child_weight=range(1,6,2)) # 97.60
#params = dict(gamma=[i/10.0 for i in range(0,5)]) # 97.68
#params = dict(subsample=[i/10.0 for i in range(6,10)], colsample_bytree=[i/10.0 for i in range(6,10)]) # 97.72
#params = dict(reg_alpha=[1e-5, 1e-2, 0.1, 1, 100]) # 97.74
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.7, gamma=0.1,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=1e-05, reg_lambda=None,
              scale_pos_weight=None, subsample=0.9, tree_method=None,
              validate_parameters=None, verbosity=None) 
Cross-Validation Score: 97.76 (+/- 0.41) [23507 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   23.5s finished


In [35]:
#!pip install lightgbm

In [36]:
# LightGBM
from lightgbm import LGBMClassifier

model = LGBMClassifier(num_leaves=73, max_depth=24, learning_rate=0.5, feature_fraction=0.75) # 97.11
evaluate_model('LGBM', model)

#params = dict(num_leaves=range(10,100), max_depth=range(1,30)) # 97.18
#params = dict(learning_rate=[0.01,0.05,0.1,0.5]) # 97.51
#params = dict(feature_fraction=[0.1,0.25,0.5,0.75,1.0]) # 97.62
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LGBMClassifier(feature_fraction=0.75, learning_rate=0.5, max_depth=24,
               num_leaves=73) 
Cross-Validation Score: 97.79 (+/- 0.46) [11912 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.9s finished


(97.78937745873812, 0.45890418436716973, 11912)

In [38]:
#!pip install catboost

In [42]:
# CatBoost
from catboost import CatBoostClassifier

model = CatBoostClassifier() # 97.04
#learning_rate=0.1, iterations=1000, depth=5
evaluate_model('CB', model)

params = dict(iterations=[10,50,100,250,500,1000],
              learning_rate=[0.01,0.05,0.1,0.5],
              depth=range(1,11,2)) 
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Cross-Validation Score: 97.04 (+/- 0.43) [45403 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   45.4s finished


In [43]:
names = []
estimators = []
scores = []
stdevs = []
times = []

for key, value in models.items():
  (model, score, stdev, elapsed) = value
  names.append(key)
  estimators.append(model)
  scores.append(score)
  stdevs.append(stdev)
  times.append(elapsed)

In [44]:
results_df = pd.DataFrame({
    'Model': names,
    'Score': scores,
    'Std Dev': stdevs,
    'Time (ms)': times,
    'Estimator': estimators})

results_df.sort_values(by='Score', ascending=False)

,Model,Score,Std Dev,Time (ms),Estimator
16,LGBM,97.789377,0.458904,11912,"LGBMClassifier(feature_fraction=0.75, learning..."
15,XGB,97.758540,0.405818,23507,"XGBClassifier(base_score=None, booster=None, c..."
17,CB,97.038814,0.429959,45403,<catboost.core.CatBoostClassifier object at 0x...
8,DT,96.514503,0.632502,199,"DecisionTreeClassifier(criterion='entropy', ma..."
10,ABDT,96.432225,0.677194,216,AdaBoostClassifier(base_estimator=DecisionTree...
11,GB,96.278037,0.422045,225645,"GradientBoostingClassifier(max_depth=35, n_est..."
9,RF,93.995493,0.421724,21692,"RandomForestClassifier(max_features='log2', n_..."
1,SVM,89.461269,0.312492,12446,"SVC(C=10, gamma=0.001, random_state=42)"
14,MLP,88.967696,0.537264,59658,"MLPClassifier(alpha=1e-07, hidden_layer_sizes=..."
3,KNN,87.466522,0.499337,3358,KNeighborsClassifier(n_neighbors=9)
